# Battle of the Neighborhoods - <font color=MediumVioletRed>Dance Studio in Thessaloniki, Greece.</font>

## Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 2)

###  <font color=blue> 1. Introduction - Discuss the business problem and who would be interested in this project. </font>

In this project we will try to find an optimal location for a Dance studio. Specifically, this report will be targeted to stakeholders interested in opening a Dance studio in Thessaloniki, Greece.

Since there are lots of Dance studios in Thessaloniki we will try to detect locations that are not already crowded with. 
We would also prefer locations as close to city center as possible.

We will use our data science powers to generate a few most promising neighborhoods based on these criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.


###  <font color=blue> 2. Data - Describe the data that will be used to solve the problem and the source of the data. </font>

Based on definition of our problem, factors that will influence our decission are:

- number of existing Dance studios in the neighborhood 
- number of and distance to Dance studios in the neighborhood
- distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:

- centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Google Maps API reverse geocoding
- number of Dance studios and their type and location in every neighborhood will be obtained using Foursquare API
- coordinate of Thessaloniki center will be obtained using Google Maps API geocoding of well known Thessaloniki location (Aristotelous Square)

###  <font color=blue> 3. Methodology section which represents the main component of the report  </font>
#### <font color=blue> Discuss and describe the exploratory data analysis, any inferential statistical testing that you performed, if any, and what machine learnings were used and why. </font>

In [1]:
# import 
import requests
import pandas as pd
import numpy as np
print ("Requests, Pandas and Numpy is imported!")

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim 

print ("All Libraries imported.")

Requests, Pandas and Numpy is imported!
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0   

###### Get the geograpical coordinate of Aristotelous Square, Thessaloniki, Greece.

In [31]:
## In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

address = 'Aristotelous Square, Thessaloniki, Greece'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Aristotelous Square, Thessaloniki, Greece are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Aristotelous Square, Thessaloniki, Greece are 40.63231705, 22.940844254956303.


###### My credentails for Foursquare.

In [6]:
# my credentials
CLIENT_ID = '2S3UUO11TYWRFRFQA0FZG0J1VFQ4VHAQPFMZIA2K212QIM0N' # your Foursquare ID
CLIENT_SECRET = '0FTALIQYOEVFOEBAGPJZAGU01ZMEA54L1NTDFAFXMR2PBJJI' # your Foursquare Secret
VERSION = '20200322' # Foursquare API version

print('My credentials are:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentials are:
CLIENT_ID: 2S3UUO11TYWRFRFQA0FZG0J1VFQ4VHAQPFMZIA2K212QIM0N
CLIENT_SECRET:0FTALIQYOEVFOEBAGPJZAGU01ZMEA54L1NTDFAFXMR2PBJJI


###### Get the url for the category Dance Studio, within  radius 5000 m.

In [10]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
category = '4bf58dd8d48988d134941735'  # define category Dance Studio
lat=latitude
lng=longitude

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    category,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=2S3UUO11TYWRFRFQA0FZG0J1VFQ4VHAQPFMZIA2K212QIM0N&client_secret=0FTALIQYOEVFOEBAGPJZAGU01ZMEA54L1NTDFAFXMR2PBJJI&v=20200322&ll=40.63231705,22.940844254956303&categoryId=4bf58dd8d48988d134941735&radius=5000&limit=100'

###### From url to pandas framework.

In [11]:
# Send the GET request and examine the resutls
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']

# pandas framework --> nearby_dance_studios
nearby_dance_studios = json_normalize(venues)

###### See nearby_dance_studios.

In [12]:
# pd.set_option('max_columns', None) # if I wanted to see all the columns
nearby_dance_studios.head()


,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,...,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4e725d41ae60854eea2aa717-0,"[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",4e725d41ae60854eea2aa717,Στρατηγού Καλλάρη 4,GR,Θεσσαλονίκη,Ελλάδα,NaN,...,"[Στρατηγού Καλλάρη 4, 546 22 Θεσσαλονίκη, Θεσσ...","[{'label': 'display', 'lat': 40.62818812865817...",40.628188,22.947086,546 22,Θεσσαλονίκη,Diagonal,0,[],66571653
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5052146fe4b054396b536a55-1,"[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",5052146fe4b054396b536a55,Βαλαωριτου 4,GR,Θεσσαλονίκη,Ελλάδα,NaN,...,"[Βαλαωριτου 4, 546 30 Θεσσαλονίκη, Θεσσαλονίκη...","[{'label': 'display', 'lat': 40.6388674884322,...",40.638867,22.938231,546 30,Θεσσαλονίκη,ThesSalsoNiki,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-503cff06e4b00cfc750b985c-2,"[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",503cff06e4b00cfc750b985c,25ης Μαρτίου 91,GR,Εύοσμος,Ελλάδα,Εθνικής Αντιστάσεως,...,"[25ης Μαρτίου 91 (Εθνικής Αντιστάσεως), Εύοσμο...","[{'label': 'display', 'lat': 40.67254886379827...",40.672549,22.918439,NaN,Θεσσαλονίκη,Vida Loca,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d4d82af9ee1370454aa848b-3,"[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",4d4d82af9ee1370454aa848b,Valaoritou 12,GR,Θεσσαλονίκη,Ελλάδα,NaN,...,"[Valaoritou 12, Θεσσαλονίκη, Θεσσαλονίκη, Ελλάδα]","[{'label': 'display', 'lat': 40.63865501943307...",40.638655,22.937599,NaN,Θεσσαλονίκη,werk:statt - Lindy Hop Greece,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5452aad3498e70afc4aaf194-4,"[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",5452aad3498e70afc4aaf194,Ερμου 46,GR,Θεσσαλονίκη,Ελλάδα,NaN,...,"[Ερμου 46, 546 23 Θεσσαλονίκη, Θεσσαλονίκη, Ελ...","[{'label': 'display', 'lat': 40.63371673693292...",40.633717,22.945195,546 23,Θεσσαλονίκη,Downtown Dance and Fashion Spot,0,[],98720561


In [13]:
# set again default
pd.reset_option('max_columns')
print('Default numbers of columns ok!')

Default numbers of columns ok!


##### Count how many Dance Studios exist. 

In [14]:
print('Total number of Dance Studios within 5000m from Aristotelous Square (Thessaloniki):', len(nearby_dance_studios))

Total number of Dance Studios within 5000m from Aristotelous Square (Thessaloniki): 23


### MAP

In [33]:
# create map of Thessaloniki using latitude and longitude values
map_thessaloniki = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, name, formattedAddress in zip(
                                            nearby_dance_studios['venue.location.lat'], 
                                            nearby_dance_studios['venue.location.lng'], 
                                            nearby_dance_studios['venue.name'], 
                                            nearby_dance_studios['venue.location.formattedAddress']):
    label = '{}, {}'.format(name, formattedAddress)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_thessaloniki)  
    
map_thessaloniki

### This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Dance Studio!

In [16]:
# clear dataframe and keep what is needed.
# rename columns needed.
nearby_dance_studios_cleared = nearby_dance_studios.rename(columns=
                                                           {'venue.id': 'venue_id', 
                                                            'venue.name': 'venue_name', 
                                                            'venue.location.address': 'address',
                                                            'venue.location.crossStreet': 'address_crossStreet',
                                                            'venue.location.formattedAddress': 'formattedAddress',
                                                            'venue.location.city': 'city',
                                                            'venue.location.postalCode': 'postalCode',
                                                            'venue.location.cc': 'country_cc',
                                                            'venue.location.country': 'country',
                                                            'venue.location.lat': 'lat',
                                                            'venue.location.lng': 'lng',
                                                            'venue.location.state': 'location_state',
                                                            'venue.location.distance' :'distance'
                                                              })
# select columns needed.
nearby_dance_studios_cleared_new=nearby_dance_studios_cleared [['venue_id',
                                                              'venue_name', 
                                                               'address',
                                                               'address_crossStreet',
                                                               'formattedAddress',
                                                               'city',
                                                               'postalCode',
                                                               'country_cc',
                                                               'lat',
                                                               'lng',
                                                               'location_state',
                                                               'distance'
                                                              ]]
nearby_dance_studios_cleared_new.head()

,venue_id,venue_name,address,address_crossStreet,formattedAddress,city,postalCode,country_cc,lat,lng,location_state,distance
0,4e725d41ae60854eea2aa717,Diagonal,Στρατηγού Καλλάρη 4,NaN,"[Στρατηγού Καλλάρη 4, 546 22 Θεσσαλονίκη, Θεσσ...",Θεσσαλονίκη,546 22,GR,40.628188,22.947086,Θεσσαλονίκη,699
1,5052146fe4b054396b536a55,ThesSalsoNiki,Βαλαωριτου 4,NaN,"[Βαλαωριτου 4, 546 30 Θεσσαλονίκη, Θεσσαλονίκη...",Θεσσαλονίκη,546 30,GR,40.638867,22.938231,Θεσσαλονίκη,761
2,503cff06e4b00cfc750b985c,Vida Loca,25ης Μαρτίου 91,Εθνικής Αντιστάσεως,"[25ης Μαρτίου 91 (Εθνικής Αντιστάσεως), Εύοσμο...",Εύοσμος,NaN,GR,40.672549,22.918439,Θεσσαλονίκη,4861
3,4d4d82af9ee1370454aa848b,werk:statt - Lindy Hop Greece,Valaoritou 12,NaN,"[Valaoritou 12, Θεσσαλονίκη, Θεσσαλονίκη, Ελλάδα]",Θεσσαλονίκη,NaN,GR,40.638655,22.937599,Θεσσαλονίκη,756
4,5452aad3498e70afc4aaf194,Downtown Dance and Fashion Spot,Ερμου 46,NaN,"[Ερμου 46, 546 23 Θεσσαλονίκη, Θεσσαλονίκη, Ελ...",Θεσσαλονίκη,546 23,GR,40.633717,22.945195,Θεσσαλονίκη,399


In [17]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [24]:

dance_studios = nearby_dance_studios_cleared_new[['lat', 'lng']]
dance_studios.head()

,lat,lng
0,40.628188,22.947086
1,40.638867,22.938231
2,40.672549,22.918439
3,40.638655,22.937599
4,40.633717,22.945195


In [30]:
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

heat_map_thessaloniki = folium.Map(location=[latitude, longitude], zoom_start=12)
heat_data = [[row['lat'],row['lng']] for index, row in dance_studios.iterrows()]

# Plot it on the map
HeatMap(heat_data).add_to(heat_map_thessaloniki)

# Display the map
heat_map_thessaloniki
